# Election Statistics ANALYSIS

## Importing Libraries

In [19]:
from pandas import DataFrame,read_csv,concat,options,set_option
from os import listdir

In [20]:
options.display.float_format = "{:.2f}".format
set_option('display.max_rows',None)

## National/State Summaries

### Load Data

In [21]:
csv_files:list[str] = listdir('State_Summary_Results')
data:dict[str,DataFrame] = {}
df:DataFrame = DataFrame(['State','KH_Vote_Count', 'KH_Vote_Pct', 'DT_Vote_Count', 'DT_Vote_Pct','Pct_Reported'])

In [22]:
for file in csv_files:
    if('FINAL' in file):
        data[file.replace('_Results.csv','').replace('FINAL_','')] = read_csv(f"State_Summary_Results/{file}")

### Merge state summaries into one giant national table

In [23]:
for key,item in data.items():
    item.insert(loc=1,column='State',value=key)
    df:DataFrame = concat([df,item],axis=0)

In [24]:
all_states_summary_df = df.drop([0],axis=1).reset_index(drop=True).iloc[6:]

### Add Nationwide Total Vote Count Columns

In [25]:
total_votes:DataFrame = all_states_summary_df.groupby('Datetime_Extraction').sum(numeric_only=True)

In [26]:
total_votes['Nationwide_Total_Votes'] = total_votes['KH_Vote_Count']+total_votes['DT_Vote_Count']
total_votes['Additional_Votes_Since_Last_Update'] = total_votes['Nationwide_Total_Votes'].diff()
# total_votes['Average']

In [27]:
total_votes[total_votes['Additional_Votes_Since_Last_Update']>0][['KH_Vote_Count','DT_Vote_Count','Nationwide_Total_Votes','Additional_Votes_Since_Last_Update']].to_csv('ANALYSIS/Nationwide_Vote_Count_Updates.csv',index=True)